## Use GridSearchCV and select the best hyperparamter for Support Vector machine

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd





In [2]:
# Importing the dataset
dataset = pd.read_csv('Advertisement.csv')


In [3]:
dataset.head(7)

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
5,15728773,Male,27,58000,0
6,15598044,Female,27,84000,0


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User ID          400 non-null    int64 
 1   Gender           400 non-null    object
 2   Age              400 non-null    int64 
 3   EstimatedSalary  400 non-null    int64 
 4   Purchased        400 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 15.8+ KB


In [5]:
dataset.isnull().sum()

User ID            0
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64

In [6]:
dataset.isnull().any()

User ID            False
Gender             False
Age                False
EstimatedSalary    False
Purchased          False
dtype: bool

In [7]:
gender=pd.get_dummies(dataset['Gender'],drop_first=True).astype(int)

In [8]:
dataset=pd.concat([gender,dataset],axis=1)

In [9]:
dataset.drop(columns=['Gender','User ID'],axis=1,inplace=True)

In [10]:
dataset['Purchased'].unique()

array([0, 1], dtype=int64)

In [11]:
dataset.head()

,Male,Age,EstimatedSalary,Purchased
0,1,19,19000,0
1,1,35,20000,0
2,0,26,43000,0
3,0,27,57000,0
4,1,19,76000,0


In [12]:
X=dataset.iloc[:,:3]

In [13]:
X

,Male,Age,EstimatedSalary
0,1,19,19000
1,1,35,20000
2,0,26,43000
3,0,27,57000
4,1,19,76000
...,...,...,...
395,0,46,41000
396,1,51,23000
397,0,50,20000
398,1,36,33000


In [14]:
y=dataset.iloc[:,3]

In [15]:
y

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [16]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 5)

In [17]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [18]:
# Fitting Kernel SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train, y_train)

SVC(kernel='linear', random_state=0)

In [19]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [20]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)


[[61  5]
 [ 9 25]]


In [21]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)

In [22]:
accuracy

0.86

In [23]:
# Applying Grid Search to find the best model and the best parameters
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [0.1,1, 10, 100, 1000], 'kernel': ['linear']},
              {'C': [0.1,1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},
             {'C':[0.1,1,10,100,1000],'kernel':['poly'],'gamma':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},
              {'C':[0.1,1,10,100,1000],'kernel':['sigmoid'],'gamma':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1,verbose=3)
grid_search = grid_search.fit(X_train, y_train)


Fitting 10 folds for each of 140 candidates, totalling 1400 fits


In [24]:
'''# Applying Grid Search to find the best model and the best parameters
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [0.1,1, 10, 100, 1000], 'kernel': ['linear']},
              {'C': [0.1,1, 10, 100, 1000], 'kernel': ['rbf','poly','gamma','sigmoid'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1,verbose=3)
grid_search = grid_search.fit(X_train, y_train)
'''

"# Applying Grid Search to find the best model and the best parameters\nfrom sklearn.model_selection import GridSearchCV\nparameters = [{'C': [0.1,1, 10, 100, 1000], 'kernel': ['linear']},\n              {'C': [0.1,1, 10, 100, 1000], 'kernel': ['rbf','poly','gamma','sigmoid'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]\ngrid_search = GridSearchCV(estimator = classifier,\n                           param_grid = parameters,\n                           scoring = 'accuracy',\n                           cv = 10,\n                           n_jobs = -1,verbose=3)\ngrid_search = grid_search.fit(X_train, y_train)\n"

In [25]:
accuracy = grid_search.best_score_


In [26]:
accuracy

0.9099999999999999

In [27]:
grid_search.best_params_

{'C': 0.1, 'gamma': 0.9, 'kernel': 'rbf'}

In [28]:
classifier = SVC(C=0.1,kernel = 'rbf', gamma=0.9,random_state=0)
classifier.fit(X_train, y_train)

SVC(C=0.1, gamma=0.9, random_state=0)

In [29]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [30]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)


[[61  5]
 [ 3 31]]


In [31]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)

In [32]:
accuracy

0.92